In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Object-Detection

Diese Übung basiert zum Teil auf: [https://pytorch.org/vision/stable/models.html#object-detection-instance-segmentation-and-person-keypoint-detection](https://pytorch.org/vision/stable/models.html#object-detection-instance-segmentation-and-person-keypoint-detection)


## Lernziele

- Object-Detection: Anwenden von Pre-Trained Modellen, Verstehen & Evaluieren der Outputs
- COCO Format: Kennenlernen, Benutzen

In [ ]:
from pathlib import Path

from IPython.display import Image 
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import seaborn as sns
import torch
from tqdm.notebook import tqdm

## Parameter

Definieren Sie den Pfad zu den Daten und extrahieren Sie die Daten.

Für Colab-Nutzer: Führen Sie folgenden Code aus um die Daten zu lesen:

"""
%%bash

!mkdir /content/data

!wget https://github.com/i4Ds/sgi-bveri-assignments-hs2022/tree/main/assignments/03_object_detection/data.tar.gz -P /content/data/ 

"""

In [ ]:
DATA_PATH = Path("./data")

In [ ]:
%%bash
ROOT_PATH=$(pwd)
tar -xvzf ${ROOT_PATH}/data.tar.gz

## Pre-Trained _Faster R-CNN_

In dieser Aufgabe werden wir ein vortrainiertes Object Detection Modell der Familie der _Faster R-CNNs_ einsetzen.


Lesen Sie die folgenden Bilder ein mit `PIL.Image`. Schauen Sie die Bilder an und überlegen Sie sich wie gut Object-Detection funktionieren könnte.

```
DATA_PATH.joinpath("dog.jpg")
DATA_PATH.joinpath("ducks.jpeg")
````

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Laden Sie ein vortrainiertes Modell der _Faster R-CNN_ Familie von [torchvision](https://pytorch.org/vision/stable/models.html#object-detection-instance-segmentation-and-person-keypoint-detection). Eine Möglichkeit ist z.B. das `fasterrcnn_mobilenet_v3_large_320_fpn`, welches Resourcen-schonend ist. Wenn Sie bessere Performance möchten, können Sie gerne ein anderes wählen.

```
from torchvision.models.detection import (
    fasterrcnn_mobilenet_v3_large_320_fpn,
    FasterRCNN_MobileNet_V3_Large_320_FPN_Weights
  )
```

Initialisieren Sie das Modell und setzten Sie es in den `eval` Mode. Setzten Sie `box_score_thresh` auf einen Wert zwischen 0.5 und 0.9.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Benutzen Sie Funktion `inference()` um Predictions für ein Bild zu generieren. Schauen Sie sich dazu folgendes Beispiel an: https://pytorch.org/vision/stable/models.html#object-detection-instance-segmentation-and-person-keypoint-detection

Erstellen Sie danach Predictions für "dog.jpg" und inspizieren Sie den Output.

In [ ]:
def inference(img, model, preprocess):
    """ Inference on a single image
        Args:
            img: (C, H, W) torch.tensor
            model: torchvision.models.detection.faster_rcnn.FasterRCNN
            preprocess: function to pre-process image batch for the model
            
        Returns:
            predictions: Dict with lists of object detections
    """
    
    image_batch = img.unsqueeze(0)
    image_processed = preprocess(image_batch)
    return model(image_processed)[0]
    

# YOUR CODE HERE
raise NotImplementedError()

Visualisieren Sie die Predictions mit [torchvision.utils.draw_bounding_boxes](torchvision.utils.draw_bounding_boxes). Visualisieren Sie die Labels, sowie die Confidence-Scores der Predictions zusammen mit den Bounding-Boxes.

Die Labels finden Sie in `weights.meta["categories"]`

In [ ]:
from torchvision.utils import draw_bounding_boxes

def draw_boxes(image, predictions, categories):
    """ Draw Boxes from Predictions 
        Args:
            image: The input image torch.tensor
            predictions: Output of inference()
            categories: List of category labels
        Returns:
            PIL.Image
    """
    labels = [f"{categories[i]} ({s * 100:.2f} %)" for i, s in zip(predictions["labels"], predictions["scores"])]

    box = draw_bounding_boxes(
        image, boxes=predictions["boxes"],
        labels=labels, width=5,
        colors="red")
    im = box.detach()
    im = Image.fromarray(im.permute(1, 2, 0).numpy())
    return im

# YOUR CODE HERE
raise NotImplementedError()

Initialisieren Sie das Modell neu und wählen Sie einen tieferen Wert für [box_score_thresh](https://github.com/pytorch/vision/blob/main/torchvision/models/detection/faster_rcnn.py). 

Erstellen Sie danach Predictions für "ducks.jpeg".

Visualisieren Sie wieder die Boxen.

Berechnen Sie die IoU für die gefundenen Boxen. Sie können folgende Funktion verwenden [torchvision.ops.box_iou](https://pytorch.org/vision/stable/generated/torchvision.ops.box_iou.html#torchvision.ops.box_iou)

In [ ]:
from torchvision.ops import box_iou

# YOUR CODE HERE
raise NotImplementedError()


Nun schauen wir uns die _activation maps_ vom Backbone-CNN an, welche in das RPN geht. Verwenden Sie dazu die folgende Funktion und inspizieren Sie die Shape der _activation map_.

Schauen Sie sich die _acitvation maps_ von beiden Beispiel-Bildern an. Was stellen Sie fest?

In [ ]:
def backbone(img, model, preprocess):
    """ Get Features from the Backbone Network
        Args:
            img: (C, H, W) torch.tensor
            model: torchvision.models.detection.faster_rcnn.FasterRCNN
            preprocess: function to pre-process image batch for the model
            
        Returns:
            predictions: Dict with lists of object detections
    """
    image_batch = img.unsqueeze(0)
    image_processed = preprocess(image_batch)
    features = model.backbone(image_processed)
    return features['0']

# YOUR CODE HERE
raise NotImplementedError()

Nun schauen wir uns den Output des RPNs an. Vergleichen Sie wieder die beiden Bilder.

Setzen Sie die Modell-Parameter: `rpn_score_thresh` und  `rpn_post_nms_top_n_test` und schauen Sie verschiedene Werte an.

In [ ]:
def rpn(img, model, preprocess):
    """ Get Region Proposals
        Args:
            img: (C, H, W) torch.tensor
            model: torchvision.models.detection.faster_rcnn.FasterRCNN
            preprocess: function to pre-process image batch for the model
            
        Returns:
            predictions: Dict with lists of object detections
    """
    image_batch = img.unsqueeze(0)
    image_processed = preprocess(image_batch)
    
    images, targets = model.transform(image_processed, targets=None)
    features = model.backbone(image_processed)
    proposals, proposal_losses = model.rpn(images, features, targets=targets)

    original_image_sizes: List[Tuple[int, int]] = []
    for img in image_batch:
        val = img.shape[-2:]
        torch._assert(
            len(val) == 2,
            f"expecting the last two dimensions of the Tensor to be H and W instead got {img.shape[-2:]}",
        )
        original_image_sizes.append((val[0], val[1]))
    proposals = model.transform.postprocess([{'boxes': proposals[0]}], images.image_sizes, original_image_sizes)

    return proposals


def draw_proposals(image, proposals):
    """ Draw Boxes from Predictions 
        Args:
            image: The input image torch.tensor
            predictions: Output of inference()
            categories: List of category labels
        Returns:
            PIL.Image
    """

    box = draw_bounding_boxes(
        image, boxes=proposals,width=5,
        colors="red")
    im = box.detach()
    im = Image.fromarray(im.permute(1, 2, 0).numpy())
    return im

# YOUR CODE HERE
raise NotImplementedError()

## Evaluation

Wir möchten nun unser Modell evaluieren. Dazu verwenden wir die "pycocotools". COCO ist ein Datensatz der Daten und Labels für verschiedene Computer Vision Tasks zur Verfügung stellt. Ausserdem definiert das COCO-Format wie Labels gespeichert werden und auch wie Modelle evaluiert werden.

https://cocodataset.org/#explore



In [ ]:
!pip install pycocotools

In [ ]:
from pycocotools.coco import COCO
import skimage.io as io
from torchvision.datasets import CocoDetection

# We define the COCO-Dateset - The train split of the 2017 competition
ds_coco = CocoDetection(
    root=DATA_PATH.joinpath("coco_images"),
    annFile=DATA_PATH.joinpath('annotations/instances_train2017.json'))

# We read the images that were prepared for this 
with open(DATA_PATH.joinpath("coco_images.txt"), "r") as f:
    coco_images_inventory = [int(x) for x in f.readlines()]

Wir wählen nun ein Beispiel Bild aus und schauen uns das COCO-Format / die Annotationen für dieses Bild an.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Wenden Sie nun das Modell auf das Beispiel-Bild an. Zeichnen Sie anschliessend die Detections ins das Bild mit der Funktion `draw_boxes`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Zeichnen Sie nun die annotierten Boxen ein. Benutzen Sie dazu die Funktion `coco_annotations_as_predictions` zum Konvertieren der COCO-Annotationen für ein Bild in dasselbe Format wie die Predictions. Dann können Sie diese mit der Funktion `draw_boxes` visualisieren.

In [ ]:
from torchvision.ops import box_convert

def coco_annotations_as_predictions(coco_annotations):
    """ Convert COCO Annotations to Predictions for Plotting """
    coco_as_predictions = {'boxes': [], 'labels': [], 'scores': []}
    for i, coco_annotation in enumerate(coco_annotations):
        coco_as_predictions['boxes'].append(coco_annotation['bbox'])
        coco_as_predictions['labels'].append(coco_annotation['category_id'])
        coco_as_predictions['scores'].append(1)

    coco_as_predictions['boxes'] = box_convert(torch.tensor(coco_as_predictions['boxes']), in_fmt='xywh', out_fmt='xyxy')
    return coco_as_predictions
# YOUR CODE HERE
raise NotImplementedError()

Berechnen Sie nun IoU zwischen Detections und Annotationen, mit der Funktion `box_iou`. Achtung: die COCO-Annotationen sind im Format 'xywh', während die Predictions im Format 'xyxy' sind. Konvertieren Sie die COCO-Annotationen zu 'xyxy'.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Predictions für mehrere Bilder.

In [ ]:
images_all = list()
for coco_image_id in tqdm(coco_images_inventory):
    #print(f"{i}/{len(coco_images_inventory)}")
    coco_image, coco_annotations = ds_coco[ds_coco.ids.index(coco_image_id)]
    image_torch = torch.tensor(np.array(coco_image)).permute(2, 0, 1)
    images_all.append(image_torch)

In [ ]:
from tqdm.notebook import tqdm

model = Model(weights=weights, box_score_thresh=0.8)
model = model.eval()

with torch.no_grad():
    predictions_all = list()
    for image_torch in tqdm(images_all):
        predictions = inference(image_torch, model, weights.transforms())
        predictions_all.append(predictions)


We now convert the Predictions to the COCO-Format for evaluation.

In [ ]:
import itertools

def convert_predictions_to_coco(predictions, image_id):
    coco_predictions_for_eval = list()
    for i,_ in enumerate(predictions['labels']):
        boxes = list(box_convert(predictions['boxes'][i], out_fmt='xywh', in_fmt='xyxy').detach().numpy())
        res = {
            "image_id": image_id,
            "category_id": int(predictions['labels'][i]),
            "bbox":boxes,"score": float(predictions['scores'][i]),
            "iscrowd": None,
            'area': float(predictions['boxes'][i][2] * predictions['boxes'][i][3])
        }
        coco_predictions_for_eval.append(res)
    return coco_predictions_for_eval

coco_predictions_for_eval = [convert_predictions_to_coco(x, image_id) for x, image_id in zip(predictions_all, coco_images_inventory)]
coco_predictions_for_eval = list(itertools.chain.from_iterable(coco_predictions_for_eval))

In [ ]:
coco_predictions_for_eval[0]

In [ ]:
from pycocotools.cocoeval import COCOeval
annType = 'bbox'
annFile=DATA_PATH.joinpath('annotations/instances_train2017.json')
coco=COCO(str(annFile))
cocoDt=coco.loadRes(coco_predictions_for_eval)

cocoEval = COCOeval(coco, cocoDt, annType)
cocoEval.params.imgIds = coco_images_inventory
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

## Appendix - Prepare Data for Exercise

In [ ]:
#%%bash
#wget https://drive.google.com/file/d/1t_l9uyBPfxSEzcajTk4a1TaQXzeRm9hw/view?usp=sharing -P `(pwd)`/data/coco

In [ ]:
annFile=DATA_PATH.joinpath('annotations/instances_train2017.json')
coco=COCO(str(annFile))

Get COCO data for specific categories:

In [ ]:
import skimage.io as io

catIds = coco.getCatIds(catNms=['dog','cat']);
imgIds = coco.getImgIds(catIds=catIds)
img = coco.loadImgs(imgIds)

annIds = coco.getAnnIds(imgIds=imgIds)
coco_images = coco.loadImgs(imgIds)

for img in coco_images:
    coco_image = Image.fromarray(io.imread(img['coco_url']))
    coco_image.save(DATA_PATH.joinpath(f"coco_images/{img['file_name']}"))

    

In [ ]:
with open(DATA_PATH.joinpath("coco_images.txt"), "w") as f:
    lines = [str(x) + "\n" for x in imgIds[:-1]] + [str(imgIds[-1])]
    f.writelines(lines)